<a href="https://colab.research.google.com/github/Salome-06/PROGCOM-A/blob/main/HQ4_STOP_Salom%C3%A9_Cubides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import os

#Definición de categorías según los niveles de dificultad, se empieza con 5 categorías y al ir subiendo el nivel se unen más
CATEGORIAS_POR_NIVEL = {
    "fácil": ["Nombre", "Comida", "Animal", "Cosa", "Color"],
    "normal": ["País o Ciudad", "Profesión", "Fruta o Verdura", "Marca o Producto", "Parte del Cuerpo"],
    "are you crazy": ["Personaje Famoso", "Objeto Abstracto", "Verbo", "Película o Serie", "Elemento o Música"],
}

#Base de datos de la máquina (ampliada ligeramente con algunos datos)
RESPUESTAS_MAQUINA_BASE = {
    "Nombre": {"M": "Mauricio", "L": "Lupita", "C": "Carlos", "A": "Ana", "P": "Pedro", "E": "Elena", "S": "Sofía", "T": "Tania", "R": "Raúl"},
    "Comida": {"M": "Manzana", "L": "Limón", "C": "Cebolla", "A": "Arroz", "P": "Papaya", "E": "Empanada", "S": "Sopa", "T": "Tostada", "R": "Ramen"},
    "Animal": {"M": "Mono", "L": "Lagarto", "C": "Cebra", "A": "Ardilla", "P": "Perro", "E": "Elefante", "S": "Serpiente", "T": "Tigre", "R": "Rana"},
    "Cosa": {"M": "Mesa", "L": "Lápiz", "C": "Cama", "A": "Almohada", "P": "Plato", "E": "Espejo", "S": "Silla", "T": "Teclado", "R": "Reloj"},
    "Color": {"M": "Morado", "L": "Lila", "C": "Celeste", "A": "Azul", "P": "Púrpura", "E": "Esmeralda", "S": "Salmón", "T": "Turquesa", "R": "Rojo"},
    "País o Ciudad": {"M": "México", "L": "Londres", "C": "Canadá", "A": "Argentina", "P": "París", "E": "Ecuador", "S": "Singapur", "T": "Tokio", "R": "Roma"},
    "Profesión": {"M": "Médico", "L": "Librero", "C": "Carpintero", "A": "Actor", "P": "Panadero", "E": "Escritor", "S": "Sociólogo", "T": "Taxista", "R": "Reportero"},
    "Fruta o Verdura": {"M": "Melón", "L": "Lulo", "C": "Ciruela", "A": "Aguacate", "P": "Pera", "E": "Espárrago", "S": "Sandía", "T": "Tomate", "R": "Rábano"},
    "Marca o Producto": {"M": "Microsoft", "L": "Lego", "C": "Canon", "A": "Adidas", "P": "Pepsi", "E": "Epson", "S": "Samsung", "T": "Taco Bell", "R": "Rolex"},
    "Parte del Cuerpo": {"M": "Mano", "L": "Lengua", "C": "Codo", "A": "Axila", "P": "Pelo", "E": "Espalda", "S": "Senos", "T": "Talón", "R": "Riñón"},
    "Personaje Famoso": {"M": "Maluma", "L": "DiCaprio", "C": "Cristiano", "A": "Adele", "P": "Piqué", "E": "Einstein", "S": "Shakespeare", "T": "Tyson", "R": "Rihanna"},
    "Objeto Abstracto": {"M": "Miedo", "L": "Lealtad", "C": "Confianza", "A": "Amor", "P": "Paz", "E": "Esperanza", "S": "Suerte", "T": "Tiempo", "R": "Rencor"},
    "Verbo": {"M": "Mirar", "L": "Luchar", "C": "Correr", "A": "Amar", "P": "Pensar", "E": "Escribir", "S": "Saltar", "T": "Tomar", "R": "Reír"},
    "Película o Serie": {"M": "Matrix", "L": "LOST", "C": "Casablanca", "A": "Alien", "P": "Pulp Fiction", "E": "Élite", "S": "Seinfeld", "T": "Titanic", "R": "Rocky"},
    "Elemento o Música": {"M": "Metal", "L": "Latín", "C": "Cumbia", "A": "Álbum", "P": "Pop", "E": "Electrónica", "S": "Salsa", "T": "Techno", "R": "Reggaetón"}
}

#Letras válidas para el juego
LETRAS_VALIDAS = [letra for letra in "ABCDEFGHIJKLMNÑOPRSTUV"]

#FUNCIONES DE LÓGICA DE JUEGO: Lo que maneja el poder jugar bien

def limpiar_pantalla():
    """Limpia la consola para una mejor visualización."""
    os.system('cls' if os.name == 'nt' else 'clear')

def obtener_respuesta_maquina(categoria, letra, nivel):
    """
    La máquina elige una respuesta. Simula dificultad: en 'crazy', la máquina puede fallar.
    """
    respuesta = RESPUESTAS_MAQUINA_BASE.get(categoria, {}).get(letra)

    #Si no hay respuesta en la base de datos, la máquina se rinde ('-')
    if not respuesta:
        return "-"

    #Simulación de fallo en nivel CRAZY (20% de probabilidad)
    if nivel == "are you crazy" and random.random() < 0.2:
        return "-"

    return respuesta.upper()

def es_respuesta_valida(respuesta, letra):
    """
    Valida la respuesta: debe empezar con la letra y tener un MÍNIMO de 3 caracteres.
    """
    if respuesta == '-':
        return False

    #**Validación Estricta**
    return respuesta.startswith(letra) and len(respuesta) >= 3

def calcular_puntuacion(respuesta_h, respuesta_b, letra):
    """
    Aplica la lógica de puntuación: 10/10 (dif), 5/5 (igual), 10/0 ó 0/10 (único).
    """
    puntos_humano = 0
    puntos_bot = 0

    #Se valida la respuesta del humano con las reglas estrictas
    es_valida_h = es_respuesta_valida(respuesta_h, letra)
    #Se asume que la respuesta de la máquina es válida si no es '-'
    es_valida_b = (respuesta_b != '-') and respuesta_b.startswith(letra)

    if es_valida_h and es_valida_b:
        if respuesta_h == respuesta_b:
            #Caso 1: Correctas E IGUALES (5 puntos c/u)
            puntos_humano = 5
            puntos_bot = 5
            mensaje = "EMPATE, misma palabra. (5 pts c/u)"
        else:
            #Caso 2: Correctas y DIFERENTES (10 puntos c/u)
            puntos_humano = 10
            puntos_bot = 10
            mensaje = "¡PUNTOS DOBLES! Diferentes. (10 pts c/u)"
    elif es_valida_h:
        #Caso 3: Solo el humano respondió correctamente (10 puntos)
        puntos_humano = 10
        mensaje = "¡PUNTAZO! Solo tú respondiste. (10 pts)"
    elif es_valida_b:
        #Caso 4: Solo la máquina respondió correctamente (10 puntos)
        puntos_bot = 10
        mensaje = "La máquina se adelantó. (10 pts)"
    else:
        #Caso 5: Nadie o ambos fallaron/inválidos
        mensaje = "Nadie o ambos fallaron/Inválidos."

    return puntos_humano, puntos_bot, mensaje


def jugar_stop():
    """Función principal que ejecuta el juego por niveles."""
    limpiar_pantalla()
    print("🎮 ¡Bienvenido al juego STOP!")

    #1. Selección de dificultad
    while True:
        dificultad = input("Elige dificultad (fácil / normal / are you crazy): ").lower().strip()
        if dificultad in CATEGORIAS_POR_NIVEL:
            break
        print("Dificultad no válida. Por favor, elige una de las opciones.")

    categorias = CATEGORIAS_POR_NIVEL[dificultad]
    letra = random.choice(LETRAS_VALIDAS)
    letra_mayus = letra.upper()

    limpiar_pantalla()
    print(f"\n--- INICIANDO RONDA: {dificultad.upper()} ---")
    print(f"🔤 LETRA SELECCIONADA: {letra_mayus}")
    print("¡Escribe '-' si no tienes respuesta. La respuesta debe tener al menos 3 letras!")

    respuestas_humano = {}

    #2. Captura de respuestas del humano
    print("\n⏳ Es tu turno. ¡Escribe rápido!")
    num_categorias = len(categorias)
    ronda_terminada_por_stop = False

    for i, categoria in enumerate(categorias, 1):
        if ronda_terminada_por_stop:
            respuestas_humano[categoria] = "-" #Rellena las categorías restantes
            continue

        while True:
            #La solicitud de respuesta es limpia y simple
            respuesta = input(f"Categoría {i}/{num_categorias} ({categoria}): ").strip().upper()

            if not respuesta:
                print("El campo no puede estar vacío. Escribe tu palabra o '-'.")
                continue

            #La validación de la primera letra y longitud
            if respuesta != '-' and not respuesta.startswith(letra_mayus):
                print(f"La palabra debe empezar con '{letra_mayus}' o ser '-'.")
                continue
            if respuesta != '-' and len(respuesta) < 3:
                print("¡Palabra muy corta! Debe tener al menos 3 letras. Intenta de nuevo.")
                continue

            respuestas_humano[categoria] = respuesta
            break # Pasa a la siguiente categoría

        # **LÓGICA DE STOP AÑADIDA AQUÍ:**
        if i == num_categorias:
            print("\n---------------------------------------------------------")
            stop_input = input("¡ÚLTIMA CATEGORÍA RESPONDIDA! ¿Dices STOP? Escribe 'STOP' o presiona ENTER para seguir: ").strip().upper()

            if stop_input == "STOP":
                ronda_terminada_por_stop = True
                print("\n🛑 ¡Has dicho STOP! Finalizando la ronda y calculando puntos...")

        elif i == num_categorias - 1:
             #Este es un buen lugar para avisar que la siguiente es la última
             pass

    #3. Generación de respuestas de la máquina (para todas las categorías)
    respuestas_bot = {}
    for categoria in categorias:
        if categoria not in respuestas_humano:
            # Si el humano dijo STOP antes de esta categoría, el bot también se rinde en esa.
            respuestas_bot[categoria] = "-"
        else:
            # Si el humano jugó la categoría, el bot genera su respuesta
            respuestas_bot[categoria] = obtener_respuesta_maquina(categoria, letra_mayus, dificultad)

    #Si se dijo STOP al final, las respuestas restantes del humano ya fueron rellenas con '-'

    #4. Puntuación
    print("\n---------------------------------------------------------")
    print("---------------------------------------------------------")

    puntos_humano_final = 0
    puntos_bot_final = 0

    print("\n📊 RESULTADOS DETALLADOS:")
    print(f"{'Categoría':<20} | {'Tú (Respuesta)':<20} | {'Máquina (Respuesta)':<20} | Ptos H | Ptos M | {'Comentario'}")
    print("-" * 95)

    for categoria in categorias:
        rh = respuestas_humano.get(categoria, '-')
        rb = respuestas_bot.get(categoria, '-')

        ph, pb, mensaje = calcular_puntuacion(rh, rb, letra_mayus)

        puntos_humano_final += ph
        puntos_bot_final += pb

        print(f"{categoria:<20} | {rh:<20} | {rb:<20} | {ph:<6} | {pb:<6} | {mensaje}")

    #5. Resultado Final
    print("-" * 95)
    print(f"🏁 Puntuación final: Tú → {puntos_humano_final} pts | Máquina → {puntos_bot_final} pts")

    if puntos_humano_final > puntos_bot_final:
        print("\n🎉 ¡GANASTE! ¡Misión cumplida! 🎉")
    elif puntos_humano_final < puntos_bot_final:
        print("\n🤖 La máquina te ha superado. ¡Fue una rival digna! 🤖")
    else:
        print("\n🤝 ¡EMPATE! Ambos son unos maestros del STOP! 🤝")

#Ejecutar el juego:
if __name__ == "__main__":
    jugar_stop()

🎮 ¡Bienvenido al juego STOP!
Elige dificultad (fácil / normal / are you crazy): fácil

--- INICIANDO RONDA: FÁCIL ---
🔤 LETRA SELECCIONADA: L
¡Escribe '-' si no tienes respuesta. La respuesta debe tener al menos 3 letras!

⏳ Es tu turno. ¡Escribe rápido!
Categoría 1/5 (Nombre): Luisa
Categoría 2/5 (Comida): Lasaña
Categoría 3/5 (Animal): Lobo
Categoría 4/5 (Cosa): Lápiz
Categoría 5/5 (Color): Lavanda

---------------------------------------------------------
¡ÚLTIMA CATEGORÍA RESPONDIDA! ¿Dices STOP? Escribe 'STOP' o presiona ENTER para seguir: STOP

🛑 ¡Has dicho STOP! Finalizando la ronda y calculando puntos...

---------------------------------------------------------
---------------------------------------------------------

📊 RESULTADOS DETALLADOS:
Categoría            | Tú (Respuesta)       | Máquina (Respuesta)  | Ptos H | Ptos M | Comentario
-----------------------------------------------------------------------------------------------
Nombre               | LUISA               